In [ ]:
! mkdir -p data/Biden data/Trump
! jq -r '.[] | [.file,.answer] | @sh' data.json | xargs -L 1 echo | parallel --will-cite --colsep ' ' wget 'https://static01.nyt.com/newsgraphics/2020/08/24/fridge-politics/assets/images/fridges/big/{1}' -O data/\{2\}/\{1\}

In [1]:
import numpy as np
import tensorflow as tf
import autokeras as ak

In [2]:
data_dir = './data'
img_height = 700
img_width = 525

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode='binary',
  image_size=(img_height, img_width),
  batch_size=1000).unbatch().shuffle(buffer_size=1000, seed=123).cache().prefetch(buffer_size=1000)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  label_mode='binary',
  image_size=(img_height, img_width),
  batch_size=1000).unbatch().shuffle(buffer_size=1000, seed=123).cache().prefetch(buffer_size=1000)

Found 429 files belonging to 2 classes.
Using 344 files for training.
Found 429 files belonging to 2 classes.
Using 85 files for validation.


In [3]:
fridge_classifier = ak.ImageClassifier(
    num_classes=2,
    multi_label=False,
    loss='binary_crossentropy',
    objective='val_accuracy',
    project_name="fridge_classifier_1",
    directory="/fridge_data"
)

fridge_classifier.fit(train_ds, validation_data=val_ds)


Trial 23 Complete [00h 03m 53s]
val_accuracy: 0.5882353186607361

Best val_accuracy So Far: 0.6117647290229797
Total elapsed time: 02h 38m 51s
INFO:tensorflow:Oracle triggered exit
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /fridge_data/fridge_classifier_1/best_model/assets


In [4]:
fridge_model = fridge_classifier.export_model()
fridge_model.save("/fridge_data/fridge_classifier_model", save_format="tf")

INFO:tensorflow:Assets written to: /fridge_data/fridge_classifier_model/assets


In [107]:
fridge_model.evaluate(val_ds, return_dict=True)

85/85 [==============================] - 2s 26ms/step - loss: 0.6858 - accuracy: 0.6118


{'loss': 0.6858354806900024, 'accuracy': 0.6117647290229797}

In [108]:
my_fridge = tf.keras.preprocessing.image.load_img('./my_fridge.jpg', target_size=(img_height, img_width))
my_fridge = tf.keras.preprocessing.image.img_to_array(my_fridge)
predicted_label = fridge_model.predict(my_fridge[np.newaxis, ...])[0][0]
print(predicted_label)
if predicted_label < 0.5:
    print('Biden')
else:
    print('Trump')

0.4989042
Biden
